In [234]:
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import tqdm as tqdm
from lxml import html
import time
import itertools

In [188]:
class HouseForSale: 
    '''
    Getting desired infos from the webpage provided for the old format webpage - without 'nove-bydleni' string in its url
    '''
    def __init__(self, link):
        '''
        Input a link in order to extract the necessary information
        '''
        self.link = link

    def getSoup(self):
        r = requests.get(self.link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'html')
    
    def getParameters(self):
        '''
        Get parameters such as price, disposition, the state of the house
        '''
        bf = self.getSoup()
        bf_table = bf.find('div',{'data-element':"detail-description"}).find('table')
        table_extract = [i.text.strip() for i in bf_table.findAll('tr')]
        df_table = pd.DataFrame(table_extract)
        df_table_split = df_table[0].str.split("\n", expand = True)
        df = pd.DataFrame(df_table_split[[0,1]])
        df.columns = ['metric', 'value']
        df = df.set_index('metric')
        return df
    
    def getMap(self):
        '''
        Get location of the property using embeded google maps longtitute and lattitude parameters
        '''
        bf = self.getSoup()
        bf_map = bf.find('div',{'id':"map"})
        x = bf_map.find('iframe')['src'].find('q=') + 2 
        y = bf_map.find('iframe')['src'].find('&key')
        location = bf_map.find('iframe')['src'][x:y]
        loc_str = location.split(',')
        loc_dict = {"lat" : [loc_str[0]], "long": [loc_str[1]]}
        loc_df = pd.DataFrame.from_dict(loc_dict, orient = 'index', columns = ['value'])
        return loc_df
    
    def getDf(self):
        '''
        Return a wide dataframe from getMap() and getParameters(), index = latitute and longtitude
        '''
        df_par = self.getParameters()
        df_map = self.getMap()
        df = pd.concat([df_par, df_map])
        df = pd.melt(df.T, id_vars = ['lat','long'])
        return df

In [368]:
class HouseForSaleNew: #In progress
    '''
    Getting desired infos from the webpage provided for the new format webpage - with 'nove-bydleni' string in url
    '''
    def __init__(self, link):
        '''
        Input a link in order to extract the necessary information
        '''
        self.link = link

    def getSoup(self):
        r = requests.get(self.link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'html')
    
    def getParameters(self): #Done
        '''
        Get parameters such as price, disposition, the state of the house
        '''
        bf = self.getSoup()
        bf_table = bf.find('div', {'id':'tabInformace'}).find('table')
        bf_table
        table_extract  = [li.text.strip() for li in list_element.findAll('tr', {'class':'cline'})]
        df_table = pd.DataFrame(table_extract)
        df_table_split = df_table[0].str.split("\n", expand = True)
        pd.DataFrame(df_table_split[[0,1]], columns = ['metric', 'value'])
        df = pd.concat(
                        [pd.DataFrame(np.array(df_table_split[[0,1]]), columns = ['metric', 'value']),
                        pd.DataFrame(np.array(df_table_split[[2,3]]), columns = ['metric', 'value'])]
                        )

        return df
    
     #Cannot really extract the locationdef getMap(self):
        '''
        Get location of the property using embeded google maps longtitute and lattitude parameters
        '''
        bf = self.getSoup()
        bf_map = bf.find('div',{'id':"map"})
        x = bf_map.find('iframe')['src'].find('q=') + 2 
        y = bf_map.find('iframe')['src'].find('&key')
        location = bf_map.find('iframe')['src'][x:y]
        loc_str = location.split(',')
        loc_dict = {"lat" : [loc_str[0]], "long": [loc_str[1]]}
        loc_df = pd.DataFrame.from_dict(loc_dict, orient = 'index', columns = ['value'])
        return loc_df
    
    def getDf(self):
        '''
        Return a wide dataframe from getMap() and getParameters(), index = latitute and longtitude
        '''
        df_par = self.getParameters()
        df_map = self.getMap()
        df = pd.concat([df_par, df_map])
        df = pd.melt(df.T, id_vars = ['lat','long'])
        return df

# Downloader class
- Manual test
- defining class

## Manual test

In [12]:
def getLinks_test(link):
    
    page = requests.get(link)
    if page.status_code == requests.codes.ok:
        wp = getSoup_test(link)
        base_url = 'https://www.bezrealitky.cz'
        links = wp.findAll('div', {'product__body'})
        return [base_url + equity.find('a')['href'] for equity in links]

    next_page_text = wp.find('ul', class_="pagination justify-content-md-end").findAll('li')[-1].text

    if next_page_text == 'Další >':
        next_page_partial = wp.find('ul', class_="pagination justify-content-md-end").findAll('li')[-1].find('a')['href']
        next_page_url = base_url + next_page_partial
        links = wp.findAll('div', {'product__body'})
        return [base_url + equity.find('a')['href'] for equity in links]
    # No more 'Next' pages, finish the script
    else:
        print('Done')

## Defining class
Manual test working - Downloader_single() working
Pagination - Downloader_multi() work in progress

In [17]:
class Downloader_single:
    '''
    Download all links real estate to initiate
    '''
    def __init__(self, link):
        '''
        Give link to extract links of given webpages
        '''
        self.link = link
    
    def getSoup(self):
        r = requests.get(self.link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'lxml')

    def getLinks(self):
        page = requests.get(self.link)
        if page.status_code == requests.codes.ok:
            wp = self.getSoup()
            base_url = 'https://www.bezrealitky.cz'
            links = wp.findAll('div', {'product__body'})
            return [base_url + equity.find('a')['href'] for equity in links]

In [18]:
Downloader_single('https://www.bezrealitky.cz/vypis/nabidka-prodej/byt/praha?_token=qNLlM7SSvDXIFrdOKHnbj0gv_-OMf7vw6yUc_-ILcFo&page=3').getLinks()

['https://www.bezrealitky.cz/nemovitosti-byty-domy/604370-nabidka-prodej-bytu-spanielova-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/596362-nabidka-prodej-bytu-merhoutova-prague',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/604185-nabidka-prodej-bytu-ucnovska',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/604354-nabidka-prodej-bytu-sanderova',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/602506-nabidka-prodej-bytu-nekvasilova',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/604357-nabidka-prodej-bytu-mantovska',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/604332-nabidka-prodej-bytu-spanielova-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/597584-nabidka-prodej-bytu-v-remizku-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/603606-nabidka-prodej-bytu-tulesicka',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/604353-nabidka-prodej-bytu-za-zameckem-praha']

In [19]:
class Downloader_multi:
    '''
    Download all links real estate to initiate
    '''
    def __init__(self, link):
        '''
        Give link to extract links of given webpages
        '''
        self.link = link
    
    def getSoup(self):
        r = requests.get(self.link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'lxml')

    def getLinks(self):
        page = requests.get(self.link)
        if page.status_code == requests.codes.ok:
            wp = self.getSoup()
            base_url = 'https://www.bezrealitky.cz'
            links = wp.findAll('div', {'product__body'})
            return [base_url + equity.find('a')['href'] for equity in links]
        
#         if next_page_text == 'Další >':
#             next_page_partial = wp.find('ul', class_="pagination justify-content-md-end").findAll('li')[-1].find('a')['href']
#             next_page_url = base_url + next_page_partial
#             getLinks(next_page_url)
#         # No more 'Next' pages, finish the script
#         else:
#             print('Done')
        
            
        

In [20]:
Downloader_single('https://www.bezrealitky.cz/vypis/nabidka-prodej/byt/praha?_token=qNLlM7SSvDXIFrdOKHnbj0gv_-OMf7vw6yUc_-ILcFo&page=63').getLinks()

[]

In [34]:
class Downloader_multi_byJan:
    '''
    Download all links real estate to initiate
    '''
    def __init__(self, link):
        '''
        Give link to extract links of given webpages
        '''
        self.link = link
    
    def getSoup(self):
            r = requests.get(link)
            r.encoding = 'UTF-8'
            return BeautifulSoup(r.text, 'lxml')

    def getSoupNext(link): 
            r = requests.get(link)
            r.encoding = 'UTF-8'
            return BeautifulSoup(r.text, 'lxml')

    def getAllPages(self):    
            wp = self.getSoup(link)
            base_url = 'https://www.bezrealitky.cz'
            test_element = wp.find('ul', class_="pagination justify-content-md-end").find('a', rel = 'next')
            pglinks = pd.Series(['https://www.bezrealitky.cz/vypis/nabidka-prodej/byt/praha'])
            while test_element != None:
                    partial_url = wp.find('ul', class_="pagination justify-content-md-end").find('a', rel = 'next')['href']
                    next_url = base_url + partial_url
                    pglinks = pglinks.append(pd.Series([next_url]), ignore_index = True)
                    wp_next = self.getSoupNext(next_url)
                    test_element = wp_next.find('ul', class_="pagination justify-content-md-end").find('a', rel = 'next')
                    wp = wp_next
            else: print("Loop is done")
            # return pglinks
        
    def getLinks(self):
            page = requests.get(link)
            if page.status_code == requests.codes.ok:
                for pg in pglinks:

                    wp = self.getSoup()
                    base_url = 'https://www.bezrealitky.cz'
                    eqlinks = wp.findAll('div', {'product__body'})
                    return [base_url_prague + equity.find('a')['href'] for equity in eqlinks]


In [110]:
Webpage = Downloader_multi_byJan('https://www.bezrealitky.cz/vypis/nabidka-prodej/byt/praha')

# Class using outer function
This class is using outside function for creating the method

In [283]:
def getSoup(link):
        r = requests.get(link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'lxml')

def getLinks(link):
        page = requests.get(link)
        if page.status_code == requests.codes.ok:
            wp = getSoup(link)
            base_url = 'https://www.bezrealitky.cz'
            links = wp.findAll('div', {'product__body'})
            links_list = list()
            for equity in links:
                    if 'https://www.bezrealitky.cz' in equity.find('a')['href']:
                        links_list.append(equity.find('a')['href'])
                    else: 
                        links_list.append(base_url + equity.find('a')['href'])
        return links_list

class Downloader_multi_byTiep:
    '''
    Download all links of real estate properties on the given website
    '''
    def __init__(self, link):
        '''
        Provide real estate webpage to extract the properties links
        '''
        self.link = link

    def getAllPages(self):
            '''
            Get all possible pages from the provided website that contains links
            '''    
            wp = getSoup(self.link)
            base_url = 'https://www.bezrealitky.cz'
            test_element = wp.find('ul', class_="pagination justify-content-md-end").find('a', rel = 'next')
            links = pd.Series(['https://www.bezrealitky.cz/vypis/nabidka-prodej/byt/praha'])
            while test_element != None:
                    partial_url = wp.find('ul', class_="pagination justify-content-md-end").find('a', rel = 'next')['href']
                    next_url = base_url + partial_url
                    links = links.append(pd.Series([next_url]), ignore_index = True)
                    wp_next = getSoup(next_url)
                    test_element = wp_next.find('ul', class_="pagination justify-content-md-end").find('a', rel = 'next')
                    wp = wp_next
            else: print("Loop is done")
            return list(links)

In [163]:
links_class = Downloader_multi_byTiep('https://www.bezrealitky.cz/vypis/nabidka-prodej/byt/praha').getAllPages()

Loop is done


In [284]:
links_properties = [getLinks(li) for li in links_class]
links_properties_unlist = list(itertools.chain(*links_properties))

In [285]:
links_properties_unlist = list(itertools.chain(*links_properties))

# Links Summary and Next steps explanation
There are two types of links provided by bezrealitky.cz. New format and old format. For each type it is essential to write a class that will extract the needed data properly. For now, we have for the old format. 

In [308]:
links_properties_unlist_old = [li for li in links_properties_unlist if 'nove-bydleni' not in li]
links_properties_unlist_new = [li for li in links_properties_unlist if 'nove-bydleni' in li]
print(np.shape(links_properties_unlist_old))
print(np.shape(links_properties_unlist_new))
print(np.shape(links_properties_unlist))

In [310]:
df_complete_sale = pd.DataFrame()
for li in links_properties_unlist_old:
    df_unit = HouseForSale(li).getDf()
    df_complete_sale = pd.concat([df_complete_sale, df_unit])

In [311]:
df_complete_sale

,lat,long,variable,value
0,50.024932,14.451379,Internet:,
1,50.024932,14.451379,Číslo inzerátu:,603779
2,50.024932,14.451379,Dispozice:,Garsoniéra
3,50.024932,14.451379,Plocha:,19 m²
4,50.024932,14.451379,Cena:,2.150.000 Kč
5,50.024932,14.451379,Město:,Praha
6,50.024932,14.451379,Městská část:,Lhotka
7,50.024932,14.451379,Typ vlastnictví:,Osobní
8,50.024932,14.451379,Typ budovy:,Panel
9,50.024932,14.451379,PENB:,G


In [370]:
HouseForSale('https://www.bezrealitky.cz/nemovitosti-byty-domy/509005-nabidka-pronajem-bytu-taboritska-praha-3').getDf()

,lat,long,variable,value
0,50.0825271,14.4562782,Internet:,
1,50.0825271,14.4562782,Číslo inzerátu:,509005
2,50.0825271,14.4562782,Dispozice:,1+kk
3,50.0825271,14.4562782,Plocha:,33 m²
4,50.0825271,14.4562782,Cena:,11.500 Kč
5,50.0825271,14.4562782,Poplatky:,3.000 Kč
6,50.0825271,14.4562782,Vratná kauce:,14.500 Kč
7,50.0825271,14.4562782,Město:,Praha
8,50.0825271,14.4562782,Městská část:,Žižkov
9,50.0825271,14.4562782,Typ vlastnictví:,Osobní


In [372]:
links_class_rent = Downloader_multi_byTiep('https://www.bezrealitky.cz/vypis/nabidka-pronajem/byt/praha').getAllPages()

Loop is done


In [388]:
links_properties_rent = [getLinks(li) for li in links_class]
links_properties_rent_unlist = list(itertools.chain(*links_properties_rent))

KeyboardInterrupt: 

In [389]:
links_properties_rent_unlist

ovitosti-byty-domy/573182-nabidka-prodej-bytu-hlavni-mesto-praha',
 'https://www.bezrealitky.cz/nove-bydleni/vzorova-nemovitost/682-green-port-strasnice/25522-novy-byt-2-kk-40-m2-p-10#utm_source=bezrealitky.cz&utm_medium=odkaz&utm_campaign=detail_inzeratu',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/592993-nabidka-prodej-bytu-generala-janouska-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/572998-nabidka-prodej-bytu-platonova-hlavni-mesto-praha',
 'https://www.bezrealitky.cz/nove-bydleni/vzorova-nemovitost/720-rezidence-italska-8/26502#utm_source=bezrealitky.cz&utm_medium=odkaz&utm_campaign=detail_inzeratu',
 'https://www.bezrealitky.cz/nove-bydleni/vzorova-nemovitost/662-na-hvezdarne-trebesin/26424-byt-5-kk-3x-terasa-praha-10#utm_source=bezrealitky.cz&utm_medium=odkaz&utm_campaign=detail_inzeratu',
 'https://www.bezrealitky.cz/nove-bydleni/vzorova-nemovitost/704-bohdalecke-kvarteto/25753-1-kk-39m2-s-balkonem-praha-4#utm_source=bezrealitky.cz&utm_medium=odkaz&utm_ca

In [390]:
links_properties_rent_unlist_old = [li for li in links_properties_rent_unlist if 'nove-bydleni' not in li]
links_properties_rent_unlist_new = [li for li in links_properties_rent_unlist if 'nove-bydleni' in li]
print(np.shape(links_properties_rent_unlist_old))
print(np.shape(links_properties_rent_unlist_new))
print(np.shape(links_properties_rent_unlist))

(461,)
(161,)
(622,)


In [393]:
links_properties_rent_unlist_old 

ka-prodej-bytu-k-brusce-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/601475-nabidka-prodej-bytu-vitkova-prague',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/601464-nabidka-prodej-bytu-ke-strasnicke-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/602138-nabidka-prodej-bytu-horni-hrdlorezska-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/597153-nabidka-prodej-bytu-brandlova-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/594850-nabidka-prodej-bytu-na-morani-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/599730-nabidka-prodej-bytu-sudejovicka-ujezd-nad-lesy',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/601452-nabidka-prodej-bytu-vita-nejedleho-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/601447-nabidka-prodej-bytu-vita-nejedleho-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/601441-nabidka-prodej-bytu-vita-nejedleho-praha',
 'https://www.bezrealitky.cz/nemovitosti-byty-domy/601429-nabidka-p

In [394]:
HouseForSale(links_properties_rent_unlist[15]).getDf()

,lat,long,variable,value
0,50.067682,14.500067,Internet:,
1,50.067682,14.500067,Číslo inzerátu:,590323
2,50.067682,14.500067,Dispozice:,2+1
3,50.067682,14.500067,Plocha:,71 m²
4,50.067682,14.500067,Cena:,5.370.000 Kč
5,50.067682,14.500067,Město:,Praha
6,50.067682,14.500067,Městská část:,Strašnice
7,50.067682,14.500067,Typ vlastnictví:,Osobní
8,50.067682,14.500067,Typ budovy:,Panel
9,50.067682,14.500067,PENB:,G


In [382]:
df_complete_rent = pd.DataFrame()
for li in links_properties_rent_unlist_old:
    df_unit = HouseForSale(li).getDf()
    df_complete_sale = pd.concat([df_complete_rent, df_unit])

In [400]:
house_1 = HouseForSale(links_properties_rent_unlist_old[10]).getDf()
house_2 = HouseForSale(links_properties_rent_unlist_old[20]).getDf()
pd.concat([house_1,house_2])

,lat,long,variable,value
0,50.0585874,14.377194,Internet:,
1,50.0585874,14.377194,Číslo inzerátu:,561292
2,50.0585874,14.377194,Dispozice:,1+kk
3,50.0585874,14.377194,Plocha:,23 m²
4,50.0585874,14.377194,Cena:,2.708.000 Kč
5,50.0585874,14.377194,Město:,Praha
6,50.0585874,14.377194,Městská část:,Jinonice
7,50.0585874,14.377194,Typ vlastnictví:,Osobní
8,50.0585874,14.377194,Typ budovy:,Panel
9,50.0585874,14.377194,Vybavenost:,Nevybavený


In [399]:
np.shape(df_complete_rent)

(0, 0)

In [223]:
df_test.set_index(['lat','long']).index.value_counts()

(50.106458, 14.659418)     23
(50.078784, 14.48633)      22
(50.041711, 14.569642)     22
(50.058553, 14.282639)     21
(50.094149, 14.495124)     20
(50.030721, 14.369716)     18
(50.053638, 14.355187)     16
(50.06744, 14.302724)      16
(50.072871, 14.477979)     15
(50.0585874, 14.377194)    15
dtype: int64